In [1]:
# Import packages 
import pandas as pd
import os
import yaml

# Local modules
from module.data_loader import OsirisData
from module.dataset import *
from module.modeling import randomforestregressormodel_train, lassoregressionmodel_train, supportvectormachinemodel_train
from module.modeling import randomforestregressormodel_pred, lassoregressionmodel_pred, supportvectormachinemodel_pred
from module.features import *
from module.settings import load_settings

In [2]:
drop_cols = ["AANW_PCT6","TOTAALAANW6","AANW_PCT10","TOTAALAANW10"]
week0 = ["AANW_PCT3","TOTAALAANW3","AANW_PCT6","TOTAALAANW6","AANW_PCT10","TOTAALAANW10"]
week3 = ["AANW_PCT6","TOTAALAANW6","AANW_PCT10","TOTAALAANW10"]
week6 = ["AANW_PCT3","TOTAALAANW3","AANW_PCT10","TOTAALAANW10"]
week10 = ["AANW_PCT3","TOTAALAANW3","AANW_PCT6","TOTAALAANW6"]

moment_selectie = week0

In [3]:
# Choose which settings to load
settings_type = 'custom'  # [default,custom] Change to 'custom' to load custom settings
dataloader = "db" # Change to 'file' for file input 

team="BA"
pred_cohort = 2025
train_cohort_min = 2023

In [4]:
config_file = 'config.yaml'
settings = load_settings(config_file, settings_type)
print(settings)

{'retrain_models': True, 'separator': ',', 'dropout_column': 'dropout', 'studentnumber_column': 'STUDENTNUMMER', 'save_method': 'xlsx', 'PROJ_ROOT': '.', 'DATA_DIR': './data', 'RAW_DATA_DIR': './data/raw', 'INTERIM_DATA_DIR': './data/interim', 'PROCESSED_DATA_DIR': './data/processed', 'EXTERNAL_DATA_DIR': './data/external', 'MODELS_DIR': './models', 'synth_data_dir_train': './data/raw/synth_data_train.csv', 'synth_data_dir_pred': './data/raw/synth_data_pred.csv', 'user_data_dir_train': './data/raw/user_data/train.csv', 'user_data_dir_pred': './data/raw/user_data/pred.csv', 'random_seed': 42, 'rf_parameters': {'bootstrap': [True, False], 'max_depth': [2, 3, 4], 'max_features': [3, 4, 5], 'min_samples_leaf': [3, 4, 5], 'min_samples_split': [2, 3, 5], 'n_estimators': [100, 200, 300]}, 'lasso_parameters': {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9]}, 'svm_parameters': {'C': [0.1, 1, 10, 100, 1000], 'gamma': [0.0001, 0.001, 0.01,

In [5]:
# Check if train.csv and pred.csv exist in user_data folder, otherwise load synthetic datasests
# When user data is loaded and an error occurs here, please check if the sep = '\t' needs to be changed in the config.yaml file to another separator like ',' or '.'
# This should be the same as the separator used in your .csv file

if dataloader == 'file':
    if os.path.exists(settings.user_data_dir_train) and os.path.exists(settings.user_data_dir_pred):
        train_path,pred_path = user_data_dir_train, user_data_dir_pred
        print ('User datasets found')
    else:
        train_path,pred_path = settings.synth_data_dir_train, settings.synth_data_dir_pred
        print ('Pre-uploaded synthetic datasets found')
        
    # train_df = pd.read_csv(train_path, sep = separator, engine='python')
    pred_df = pd.read_csv(pred_path, sep = separator, engine='python')
elif dataloader == 'db':
    db = OsirisData(env_path="./")
    # train_df = db.get_dataset(team="BB",min_cohort=train_cohort_min,max_cohort=pred_cohort-1)
    pred_df = db.get_dataset(team=team,min_cohort=pred_cohort,max_cohort=pred_cohort)
drp_cols = [x for x in moment_selectie if x in pred_df.columns]
pred_df = pred_df.drop(columns=drp_cols, errors='ignore')

__ Init done __
inschrijvingen geladen...
verzuimdata geladen...
vooropleidingdata geladen...


In [6]:
pred_df.columns

Index(['STUDENTNUMMER', 'STUD_GENDER_M', 'STUD_GENDER_V', 'STUD_GENDER_O',
       'LEEFTIJD', 'SOPL_LW_BOL', 'SOPL_LW_BBL', 'SOPL_NIV1', 'SOPL_NIV2',
       'SOPL_NIV3', 'SOPL_NIV4', 'VOOROPLNIVEAU_NAN', 'VOOROPLNIVEAU_HAVO',
       'VOOROPLNIVEAU_VMBO_BB', 'VOOROPLNIVEAU_VMBO_GL',
       'VOOROPLNIVEAU_VMBO_KB', 'VOOROPLNIVEAU_VMBO_TL', 'VOOROPLNIVEAU_VWO',
       'VOOROPLNIVEAU_MBO', 'VOOROPLNIVEAU_HO', 'TEAM', 'dropout'],
      dtype='object')

In [7]:
if settings.retrain_models == True:
    if dataloader == 'file':
        train_df = pd.read_csv(train_path, sep = separator, engine='python')
    elif dataloader == 'db':
        # pred_cohort = 2024
        # train_cohort_min = 2023
        # db = OsirisData(env_path="./")
        train_df = db.get_dataset(team=team,min_cohort=train_cohort_min,max_cohort=pred_cohort-1)
    drp_cols = [x for x in moment_selectie if x in train_df.columns]
    train_df = train_df.drop(columns=drp_cols, errors='ignore')
        # pred_df = db.get_dataset(team="BB",min_cohort=pred_cohort,max_cohort=pred_cohort)

inschrijvingen geladen...
verzuimdata geladen...
vooropleidingdata geladen...


In [8]:
# Basic data cleaning: drop rows that are duplicate and change any NA values to the average value of the column it's in. 
if settings.retrain_models == True:
    train_basic_cl = basic_cleaning (train_df)
pred_basic_cl = basic_cleaning (pred_df)

In [9]:
# Detect if there are columns in which all rows have the same value and delete these columns from the train and predict datasets 
if settings.retrain_models == True:
    train_cleaned, pred_cleaned = remove_single_value_columns(train_basic_cl, pred_basic_cl)
else: 
    train_cleaned, pred_cleaned = remove_single_value_columns(pred_basic_cl, pred_basic_cl)

In [10]:
train_cleaned[settings.studentnumber_column]=train_cleaned[settings.studentnumber_column].astype("int")
pred_cleaned[settings.studentnumber_column]=pred_cleaned[settings.studentnumber_column].astype("int")

In [11]:
# Apply function that changes categorical data into numerical data so it can be used as input for the models 
train_processed, pred_processed = convert_categorical_to_dummies(train_cleaned, pred_cleaned, settings.dropout_column, settings.separator)

In [12]:
# Use the function standardize_min_max to standardize the train and pred datasets using a min max scaler and save them as .csv files in the folder data/interim 
# These datasets can be used for the lasso and svm models, because reggression is sensitive to scaling 
train_df_sdd, pred_df_sdd = standardize_dataset(train_processed, pred_processed, settings.dropout_column, settings.separator)

In [13]:
# Code checks if retrain_models = True or False in config.yaml file. 
# When using your own datasets, change retrain_models in the config.yaml file to True, so the models are trained on your own data. 
# Warning: training the models can take a long time depending on the size and contents of your data. 
if settings.retrain_models == True:
    print ('Training models on the data...')
    best_rf_model = randomforestregressormodel_train(
        train_processed, settings.random_seed
        , settings.dropout_column, settings.rf_parameters)
    best_lasso_model = lassoregressionmodel_train(
        train_df_sdd, settings.random_seed
        , settings.dropout_column, settings.lasso_parameters)
    best_svm_model = supportvectormachinemodel_train(
        train_df_sdd, settings.random_seed
        , settings.dropout_column, settings.svm_parameters)
else:
    print('retrain_models is False in the config.yaml file, loading the pre-trained models')
# Folds = number of train/test splits of the dataset, candidates = models with different parameters and fits = folds * candidates

Training models on the data...
Training random_forest_regressor
Fitting 5 folds for each of 486 candidates, totalling 2430 fits
Training lasso_regression
Fitting 5 folds for each of 19 candidates, totalling 95 fits
Training support_vector_machine
{'probability': True}
Fitting 5 folds for each of 25 candidates, totalling 125 fits


In [14]:
train_processed.columns

Index(['STUD_GENDER_M', 'STUD_GENDER_V', 'LEEFTIJD', 'SOPL_LW_BOL',
       'SOPL_LW_BBL', 'SOPL_NIV3', 'SOPL_NIV4', 'VOOROPLNIVEAU_NAN',
       'VOOROPLNIVEAU_HAVO', 'VOOROPLNIVEAU_VMBO_BB', 'VOOROPLNIVEAU_VMBO_GL',
       'VOOROPLNIVEAU_VMBO_KB', 'VOOROPLNIVEAU_VMBO_TL', 'VOOROPLNIVEAU_MBO',
       'dropout'],
      dtype='object')

In [15]:
pred_processed.columns

Index(['STUDENTNUMMER', 'STUD_GENDER_M', 'STUD_GENDER_V', 'LEEFTIJD',
       'SOPL_LW_BOL', 'SOPL_LW_BBL', 'SOPL_NIV3', 'SOPL_NIV4',
       'VOOROPLNIVEAU_NAN', 'VOOROPLNIVEAU_HAVO', 'VOOROPLNIVEAU_VMBO_BB',
       'VOOROPLNIVEAU_VMBO_GL', 'VOOROPLNIVEAU_VMBO_KB',
       'VOOROPLNIVEAU_VMBO_TL', 'VOOROPLNIVEAU_MBO', 'dropout'],
      dtype='object')

In [16]:
# Use the loaded models to predict on the datasets. 
# The lasso and SVM models use the standardized dataset ot predict an, but take the student numnbers from the 
# regular predict dataset. 
ranked_students_rf = randomforestregressormodel_pred (pred_processed, settings.dropout_column, settings.studentnumber_column)
ranked_students_lasso = lassoregressionmodel_pred(pred_df_sdd, pred_processed, settings.dropout_column, settings.studentnumber_column)
ranked_students_svm = supportvectormachinemodel_pred(pred_df_sdd, pred_processed, settings.dropout_column, settings.studentnumber_column)

In [17]:
# Save the output files as either .xlsx or as three .csv files
from pathlib import Path
# __file__

BASE_PATH = Path(os.getcwd()).parent
DATA_PATH = BASE_PATH / 'data'
OUT_PATH = DATA_PATH / 'output'
# BASE_PATH = Path("Z:\\FRITS\\DGO-WG3")
DATA_PATH

WindowsPath('//dev-report-01/EXPORT/FRITS/Python/DGO-WG3/data')

In [19]:

if settings.save_method == 'xlsx':
    writer = pd.ExcelWriter(OUT_PATH / f'{team}_ranked_students.xlsx', engine='xlsxwriter')
    ranked_students_rf.to_excel(writer, sheet_name='Random Forest', startrow=0, startcol=0, index=False)
    ranked_students_lasso.to_excel(writer, sheet_name='Lasso', startrow=0, startcol=0, index=False)
    ranked_students_svm.to_excel(writer, sheet_name='Support Vector Machine', startrow=0, startcol=0, index=False)
    writer.close()
    print (f'Output file saved as .xlsx in the {OUT_PATH} folder')
elif settings.save_method == 'csv':
    ranked_students_rf.to_csv(OUT_PATH / f'{team}_ranked_students_rf.csv', sep='\t', index=False)
    ranked_students_lasso.to_csv(OUT_PATH / f'{team}_ranked_students_lasso.csv', sep='\t', index=False)
    ranked_students_svm.to_csv(OUT_PATH / f'{team}_ranked_students_svm.csv', sep='\t', index=False)
    print (f'Output files saved as .csv in the {OUT_PATH} folder')
else:
    print('Invalid save method. For save_method in the config.yaml file, please fill in "xlsx" or "csv"')

Output file saved as .xlsx in the \\dev-report-01\EXPORT\FRITS\Python\DGO-WG3\data\output folder


In [20]:
# %pip install seaborn

In [21]:
from module.plot import (
    generate_precision_plot, 
    generate_sensitivity_plot, 
    generate_svm_importance_plot,
    generate_stoplight_evaluation,
    save_model_metrics,
    save_threshold_analysis,
    extract_model_data,
    sort_and_filter_data,
    process_evaluation_results,
    display_model_results,
    get_coefficient_table,
    get_top_svm_features,
    analyze_missing_data,
    parse_model_metrics,
    display_top_features
)

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot styling
sns.set_theme(style="whitegrid")
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['axes.formatter.useoffset'] = False
plt.rcParams['axes.formatter.limits'] = (-2, 2)
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1.2)

### Validatie

In [ ]:
#| label: stoplight-eval
#| output: asis
#| echo: false
#| warning: false
from sklearn.model_selection import train_test_split
import warnings
# from sklearn.model_selection import train_test_split
# from sklearn import tree
# from contextlib import redirect_stdout, redirect_stderr
warnings.filterwarnings('ignore')


try:
    # --- 1. Train/test split ---
    # Use processed data for RF, scaled for Lasso/SVM
    X_train, X_test, y_train, y_test = train_test_split(
        train_processed.drop(settings.dropout_column, axis=1),
        train_processed[settings.dropout_column],
        test_size=0.2,
        random_state=42
    )
    X_train_sdd, X_test_sdd, y_train_sdd, y_test_sdd = train_test_split(
        train_df_sdd.drop(settings.dropout_column, axis=1),
        train_df_sdd[settings.dropout_column],
        test_size=0.2,
        random_state=42
    )
    # Prepare test sets for evaluation
    test_data_rf = pd.concat([X_test, y_test], axis=1)
    test_data_sdd = pd.concat([X_test_sdd, y_test_sdd], axis=1)

    # Make test data available globally for plotting functions
    globals()['test_data_rf'] = test_data_rf
    globals()['test_data_sdd'] = test_data_sdd

    # --- 2. Train or load models ---
    # if settings.retrain_models:
    #     # Suppress GridSearchCV output during training
    #     with open(os.devnull, 'w') as fnull:
    #         with redirect_stdout(fnull), redirect_stderr(fnull):
    #             best_rf_model = randomforestregressormodel_train(pd.concat([X_train, y_train], axis=1), random_seed, dropout_column, rf_parameters)
    #             best_lasso_model = lassoregressionmodel_train(pd.concat([X_train_sdd, y_train_sdd], axis=1), random_seed, dropout_column, alpha_range)
    #             best_svm_model = supportvectormachinemodel_train(pd.concat([X_train_sdd, y_train_sdd], axis=1), random_seed, dropout_column, svm_parameters)
    # else:
    #     best_rf_model = joblib.load('models/random_forest_regressor.joblib')
    #     best_lasso_model = joblib.load('models/lasso_regression.joblib')
    #     best_svm_model = joblib.load('models/support_vector_machine.joblib')

    # --- 3. Modular stoplight evaluation on test set ---
    model_predictions = {
        'Random Forest': (test_data_rf, best_rf_model, False),
        'Lasso': (test_data_sdd, best_lasso_model, True),
        'SVM': (test_data_sdd, best_svm_model, True)
    }
    
    evaluation_results = generate_stoplight_evaluation(
        model_predictions,
        invite_pct=20
    )

    # --- 4. Save metrics and threshold analysis on test set ---
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        save_model_metrics(
            train_data=pd.concat([X_train, y_train], axis=1),
            train_data_scaled=pd.concat([X_train_sdd, y_train_sdd], axis=1),
            validation_data=test_data_rf,
            validation_data_scaled=test_data_sdd,
            rf_model=best_rf_model,
            lasso_model=best_lasso_model,
            svm_model=best_svm_model
        )
        save_threshold_analysis(
            train_data=pd.concat([X_train, y_train], axis=1),
            train_data_scaled=pd.concat([X_train_sdd, y_train_sdd], axis=1),
            validation_data=test_data_rf,
            validation_data_scaled=test_data_sdd,
            rf_model=best_rf_model,
            lasso_model=best_lasso_model,
            svm_model=best_svm_model
        )
except Exception as e:
    print(f"Error in stoplight evaluation: {e}")
    import traceback
    traceback.print_exc()

# Process evaluation results using imported functions
model_results, best_model, best_metrics, recommendation_display, recommendation_text = process_evaluation_results(evaluation_results)

# print(recommendation_display)
txt = "\n\n" + recommendation_text + " Het best presterende model is: \n\n" + best_model + "\n\nHet laat de beste balans zien tussen precisie en recall bij een selectie van 20% van de studenten voor uitnodiging. " +  "Dit impliceert dat dit model het meest effectief is in het identificeren van studenten met een verhoogd risico op uitval."

In [23]:
txt_precision = f" - Precisie: {best_metrics['precision']:.1f}%\n"
txt_recall = f"- Recall: {best_metrics['recall']:.1f}%\n"
txt_status = f"- Status: {best_metrics['status']}\n"
txt_summary = f"\n**Samenvatting:**\n"
tbl_summary = best_metrics['dutch_summary']
txt2 = "\n".join([txt_precision, txt_recall, txt_status, txt_summary, tbl_summary])

In [24]:
# %pip install git+https://github.com/innovationOUtside/fstring-magic.git

In [25]:
%reload_ext fstring_magic

# Analyse

In [26]:
%%fstring
__{recommendation_display}__

{txt}

__🔴 🔴 🔴 Niet bruikbaar 🔴 🔴 🔴__



Op basis van de evaluatie kan het model NIET worden gebruikt. Het best presterende model is: 

SVM

Het laat de beste balans zien tussen precisie en recall bij een selectie van 20% van de studenten voor uitnodiging. Dit impliceert dat dit model het meest effectief is in het identificeren van studenten met een verhoogd risico op uitval.


<div class='disclaimer' style='padding: 15px; margin: 20px 0; border-left: 4px solid #ffc107;'>
<strong>Belangrijke opmerking:</strong>
<p>Deze evaluatie is automatisch gegenereerd en dient als richtlijn. De uiteindelijke beslissing over het gebruik van het model ligt bij de gebruiker. Het is belangrijk om:</p>
<ul>
      <li>De resultaten kritisch te evalueren in de context van uw specifieke situatie</li>
      <li>De beperkingen van het model te begrijpen</li>
      <li>De ethische implicaties van het gebruik te overwegen</li>
      <li>De resultaten te valideren met domeinexperts</li>
</ul>

<p>De gebruiker is zelf verantwoordelijk voor de interpretatie en het gebruik van de modelresultaten.</p>
</div>

<p>Voor meer informatie over de gebruikte modellen wordt verwezen naar Hoofdstuk 2.</p>
<p>Verdere details over de precisie, recall en model-specifieke analyses zijn te vinden in de latere hoofdstukken van dit rapport.</p>
<p/>
<h2>Prestaties van het aanbevolen model:</h2>

In [27]:
%%fstring
{txt2}

 - Precisie: 72.4%

- Recall: 26.9%

- Status: Niet bruikbaar


**Samenvatting:**

Bij 20% uitgenodigde studenten (29 uit 148 studenten):
- 26.9% van alle uitvallers wordt geïdentificeerd (21 van 78 uitvallers)
- 72.4% van de uitgenodigde studenten valt daadwerkelijk uit (21 van 29 uitgenodigde studenten)


## **Toelichting:**
- **Precisie (%):** Percentage van de uitgenodigde studenten dat daadwerkelijk uitvalt
- **Recall (%):** Percentage van alle uitvallers dat wordt geïdentificeerd
- **% Uitgenodigd:** Percentage van alle studenten dat wordt uitgenodigd

## Wat te controleren bij een slechte aanbeveling

Wanneer een model een slechte aanbeveling doet, zijn er verschillende onderdelen in dit rapport die u kunt raadplegen om mogelijke oorzaken te achterhalen. Begin bij 3.3 Model Metrics om te controleren op tekenen van overfitting of underfitting, zoals grote verschillen tussen training en testresultaten of een negatieve R².

Bekijk vervolgens hoofdstuk 4 (Data Kwaliteit) om te zien of er sprake is van ontbrekende of onbetrouwbare data die het model kan hebben beïnvloed.

Tot slot bieden hoofdstukken 5, 6 en 7 inzicht in de prestaties en relevantie van individuele features. Een lage bijdrage of onverwacht gedrag van belangrijke variabelen kan verklaren waarom het model geen goede voorspelling doet.

# Gedetailleerde Modellen

## Random Forest

In [28]:
%%fstring
{display_model_results(model_results, 'Random Forest')}

🔴 🔴 🔴 Niet bruikbaar 🔴 🔴 🔴

**Precisie:** 62.1%
**Recall:** 23.1%
**Status:** Niet bruikbaar
**Evaluatie:** Model heeft verbetering nodig

**Samenvatting:**
Bij 20% uitgenodigde studenten (29 uit 148 studenten):
- 23.1% van alle uitvallers wordt geïdentificeerd (18 van 78 uitvallers)
- 62.1% van de uitgenodigde studenten valt daadwerkelijk uit (18 van 29 uitgenodigde studenten)

**Prestaties bij Verschillende Uitnodigingspercentages**

| % Uitgenodigd | Precisie (%) | Recall (%) |
|:-------------:|:------------:|:----------:|
|         5.0 |        71.4 |        6.4 |
|        10.0 |        71.4 |       12.8 |
|        15.0 |        63.6 |       17.9 |
|        20.0 |        62.1 |       23.1 |
|        25.0 |        67.6 |       32.1 |
|        30.0 |        68.2 |       38.5 |
|        40.0 |        64.4 |       48.7 |
|        50.0 |        60.8 |       57.7 |
|        60.0 |        59.1 |       66.7 |
|        70.0 |        57.3 |       75.6 |
|        80.0 |        55.9 |       84.6 |
|        90.0 |        54.9 |       93.6 |
|       100.0 |        52.7 |      100.0 |



## Lasso

In [29]:
%%fstring 
{display_model_results(model_results, 'Lasso')}

🔴 🔴 🔴 Niet bruikbaar 🔴 🔴 🔴

**Precisie:** 55.2%
**Recall:** 20.5%
**Status:** Niet bruikbaar
**Evaluatie:** Model heeft verbetering nodig

**Samenvatting:**
Bij 20% uitgenodigde studenten (29 uit 148 studenten):
- 20.5% van alle uitvallers wordt geïdentificeerd (16 van 78 uitvallers)
- 55.2% van de uitgenodigde studenten valt daadwerkelijk uit (16 van 29 uitgenodigde studenten)

**Prestaties bij Verschillende Uitnodigingspercentages**

| % Uitgenodigd | Precisie (%) | Recall (%) |
|:-------------:|:------------:|:----------:|
|         5.0 |        42.9 |        3.8 |
|        10.0 |        57.1 |       10.3 |
|        15.0 |        63.6 |       17.9 |
|        20.0 |        55.2 |       20.5 |
|        25.0 |        59.5 |       28.2 |
|        30.0 |        63.6 |       35.9 |
|        40.0 |        61.0 |       46.2 |
|        50.0 |        64.9 |       61.5 |
|        60.0 |        58.0 |       65.4 |
|        70.0 |        56.3 |       74.4 |
|        80.0 |        55.9 |       84.6 |
|        90.0 |        54.1 |       92.3 |
|       100.0 |        52.7 |      100.0 |



## SVM

In [30]:
%%fstring 
{display_model_results(model_results, 'SVM')}

🔴 🔴 🔴 Niet bruikbaar 🔴 🔴 🔴

**Precisie:** 72.4%
**Recall:** 26.9%
**Status:** Niet bruikbaar
**Evaluatie:** Model heeft verbetering nodig

**Samenvatting:**
Bij 20% uitgenodigde studenten (29 uit 148 studenten):
- 26.9% van alle uitvallers wordt geïdentificeerd (21 van 78 uitvallers)
- 72.4% van de uitgenodigde studenten valt daadwerkelijk uit (21 van 29 uitgenodigde studenten)

**Prestaties bij Verschillende Uitnodigingspercentages**

| % Uitgenodigd | Precisie (%) | Recall (%) |
|:-------------:|:------------:|:----------:|
|         5.0 |        71.4 |        6.4 |
|        10.0 |        78.6 |       14.1 |
|        15.0 |        68.2 |       19.2 |
|        20.0 |        72.4 |       26.9 |
|        25.0 |        75.7 |       35.9 |
|        30.0 |        77.3 |       43.6 |
|        40.0 |        67.8 |       51.3 |
|        50.0 |        64.9 |       61.5 |
|        60.0 |        60.2 |       67.9 |
|        70.0 |        59.2 |       78.2 |
|        80.0 |        55.1 |       83.3 |
|        90.0 |        53.4 |       91.0 |
|       100.0 |        52.7 |      100.0 |

